In [1]:
import time
start = time.time()
import random
import string
import pandas as pd
import copy
#import os
#os.chdir('/home/scott/Documents/EDx/MIT/Python/Spyder_scripts/')
WORDLIST_FILENAME = "words.txt"

def loadWords():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # line: string
    line = inFile.readline()
    # wordlist: list of strings
    wordlist = line.split()
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def chooseWord(wordlist):
    """
    wordlist (list): list of words (strings)

    Returns a word from wordlist at random
    """
    return random.choice(wordlist)

# end of helper code
# -----------------------------------

# Load the list of words into the variable wordlist
# so that it can be accessed from anywhere in the program

def isWordGuessed(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: boolean, True if all the letters of secretWord are in lettersGuessed;
      False otherwise
    '''
    for char in secretWord:
        if char not in lettersGuessed:
            return False
    
    return True  


def getGuessedWord(secretWord, lettersGuessed):
    '''
    secretWord: string, the word the user is guessing
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters and underscores that represents
      what letters in secretWord have been guessed so far.
    '''
    # FILL IN YOUR CODE HERE...
    word = secretWord
 
    for char in word:
        if char not in lettersGuessed:
            word = word.replace(char,'_')
            
    return word
    


def getAvailableLetters(lettersGuessed):
    '''
    lettersGuessed: list, what letters have been guessed so far
    returns: string, comprised of letters that represents what letters have not
      yet been guessed.
    '''
    # FILL IN YOUR CODE HERE...
    availableLetters = 'abcdefghijklmnopqrstuvwxyz'
    for char in lettersGuessed:
            position = availableLetters.index(char)
            availableLetters = availableLetters[0:position] + availableLetters[position+1::]
    return availableLetters

wordList = loadWords()
columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13']
secrets = pd.DataFrame()
    
for i in range(1,max(len(word) for word in wordList)+1):
    dat = []
    for word in wordList:
        if len(word) == i:
            dat.append(word)
    dat = pd.Series(dat)
    secrets = pd.concat([secrets,dat],axis=1, join='outer', join_axes=None, ignore_index=False)
secrets.columns = columns

def hangman():
    '''
    secretWord: string, the secret word to guess.

    Starts up an interactive game of Hangman.

    * At the start of the game, let the user know how many 
      letters the secretWord contains.

    * Ask the user to supply one guess (i.e. letter) per round.

    * The user should receive feedback immediately after each guess 
      about whether their guess appears in the computers word.

    * After each round, you should also display to the user the 
      partially guessed word so far, as well as letters that the 
      user has not yet guessed.

    Follows the other limitations detailed in the problem write-up.
    '''
    
    n = 1
    gamesWon = 0
    while n <= 100:    
        secretWord = chooseWord(wordList)
        word = secretWord
        choices = secrets[str(len(word))]
        numberOfGuesses = 8
        print('Welcome to the game Hangman!')
        print('I am thinking of a word that is ' + str(len(word)) + ' letters long.')
        print('-------------')
        lettersGuessed = ''
        game = True
        
        while game == True:
            
                
            availableLetters = getAvailableLetters(lettersGuessed)
            print('You have ' + str(numberOfGuesses) + ' guesses left.')
            print('Available letters: ' + availableLetters)
                
            word = getGuessedWord(secretWord, lettersGuessed)
            #letterGuessed = player(secretWord, lettersGuessed)
            [letterGuessed,words] = playerAI(secretWord, lettersGuessed,word,choices)
            choices = words
                
            lettersGuessed = lettersGuessed[0::] + letterGuessed
            word = getGuessedWord(secretWord, lettersGuessed)
                    
            if letterGuessed not in secretWord:
                print("Oops! That letter is not in my word: " + word)
                print('-------------')
                numberOfGuesses -= 1
                    
            else:
                print('Good guess: ' + word)
                print('-------------')
               
            answer = isWordGuessed(secretWord,lettersGuessed)
        
            if numberOfGuesses == 0 or answer == True:
                game = False    
            
            
        if answer == True:
            print('Congratulations, you won!')
            gamesWon += 1
            n += 1
            
        else:
            print('Sorry, you ran out of guesses. The word was ' + secretWord)
            n += 1
            
    print('Player function won ' + str(gamesWon) + ' games.')
    print('Player function lost ' + str(n - gamesWon - 1) + ' games.')
    return n

# When you've completed your hangman function, uncomment these two lines
# and run this file to test! (hint: you might want to pick your own
# secretWord while you're testing)

def playerAI(secretWord, lettersGuessed,word, choices):

    availableLetters = getAvailableLetters(lettersGuessed)
    col = str(len(secretWord))
    words = secrets[col].dropna()
    words = list(words)
    options = copy.deepcopy(words)

    if len(availableLetters) < 26:
        for chance in words:
            for j in range(0,len(chance)):
                if word[j] != '_' and  word[j] != chance[j]:
                    options.remove(chance)
                    break       
            
    best = 0
    choice = ''
    for letter in availableLetters:
        
        maximum = 0
        for word in options:
            for char in word:
                if char == letter:
                    maximum += 1
        if maximum > best:
            best = maximum
            choice = letter

    return [choice,options]
    

def player(secretWord, lettersGuessed):
    
    index = random.choice(range(0,len(string.ascii_lowercase) - len(lettersGuessed)  , 1))
    availableLetters = getAvailableLetters(lettersGuessed)
    letter = availableLetters[index]
    
    return letter

n = hangman()
end = time.time()
print('PlayerAI played ' + str(n-1) + ' games in '+ str(round((end-start)/60,2)) + ' minutes')

Loading word list from file...
('  ', 55909, 'words loaded.')
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _e_____
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _e____s
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: _er___s
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: _er___s
-------------
You have 7 guesses left.
Available letters: abcdfghjklmnopqtuvwxyz
Good guess: _era__s
-------------
You have 7 guesses left.
Available letters: bcdfghjklmnopqtuvwxyz
Good guess: _era_ds
-------------
You have 7 guesses left.
Available letters: bcfghjklmnopqtuvwxyz
Good guess: hera_ds
-------------
You have 7 guesses left.
Available letters: bcfgjklmnopqtuvwxyz
Good guess: heralds
-------------
Congratula

Oops! That letter is not in my word: _________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ______i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: ______in_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqrtuvwxyz
Good guess: ______ing
-------------
You have 6 guesses left.
Available letters: abcdfhjklmopqrtuvwxyz
Oops! That letter is not in my word: ______ing
-------------
You have 5 guesses left.
Available letters: abcdfhjklmopqtuvwxyz
Good guess: __t___ing
-------------
You have 5 guesses left.
Available letters: abcdfhjklmopquvwxyz
Good guess: __t_a_ing
-------------
You have 5 guesses left.
Available letters: bcdfhjklmopquvwxyz
Good guess: _ut_a_ing
-------------
You have 5 guesses left.
Available letters: bcdfhjklmopqvwxyz
Good guess: out_a_ing
-------------
You have 5 guesses left.
Available letters: bcdfhjklmpqvwxyz
Good guess: out_acing
-------------
You h

Oops! That letter is not in my word: _______
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: _______
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: _______
-------------
You have 5 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: a__a___
-------------
You have 5 guesses left.
Available letters: bcdfghjklmnopqrtuvwxyz
Good guess: a__all_
-------------
You have 5 guesses left.
Available letters: bcdfghjkmnopqrtuvwxyz
Oops! That letter is not in my word: a__all_
-------------
You have 4 guesses left.
Available letters: bcdfghjkmnoqrtuvwxyz
Good guess: a__ally
-------------
You have 4 guesses left.
Available letters: bcdfghjkmnoqrtuvwxz
Good guess: a_rally
-------------
You have 4 guesses left.
Available letters: bcdfghjkmnoqtuvwxz
Good guess: aurally
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am th

Good guess: _a___ities
-------------
You have 8 guesses left.
Available letters: bcdfghjklmnopqruvwxyz
Oops! That letter is not in my word: _a___ities
-------------
You have 7 guesses left.
Available letters: bdfghjklmnopqruvwxyz
Oops! That letter is not in my word: _a___ities
-------------
You have 6 guesses left.
Available letters: bdfghjkmnopqruvwxyz
Oops! That letter is not in my word: _a___ities
-------------
You have 5 guesses left.
Available letters: bdghjkmnopqruvwxyz
Good guess: ma___ities
-------------
You have 5 guesses left.
Available letters: bdghjknopqruvwxyz
Good guess: maj__ities
-------------
You have 5 guesses left.
Available letters: bdghknopqruvwxyz
Good guess: majo_ities
-------------
You have 5 guesses left.
Available letters: bdghknpqruvwxyz
Good guess: majorities
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 7 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz


Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: i____i__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: i___ni__
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqrtuvwxyz
Good guess: i___nit_
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqruvwxyz
Good guess: i___nity
-------------
You have 6 guesses left.
Available letters: abcdfghjklmopqruvwxz
Good guess: imm_nity
-------------
You have 6 guesses left.
Available letters: abcdfghjklopqruvwxz
Good guess: immunity
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 9 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _

...

Good guess: jing_istic
-------------
You have 7 guesses left.
Available letters: abdfhklmopqruvwxyz
Good guess: jingoistic
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 10 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _______e__
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: _______ess
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ______ness
-------------
You have 8 guesses left.
Available letters: abcdfghijklmopqrtuvwxyz
Good guess: _i___iness
-------------
You have 8 guesses left.
Available letters: abcdfghjklmopqrtuvwxyz
Good guess: _il__iness
-------------
You have 8 guesses left.
Available letters: abcdfghjkmopqrtuvwxyz
Oops! That letter is not in my word: _il__iness
-------------
You have 7 guesses left.
Available letters: bcdfghjkmopqrtuvwxyz
Good guess: fil__iness

Good guess: stiller
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 6 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: _____e
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: __a__e
-------------
You have 8 guesses left.
Available letters: bcdfghijklmnopqrstuvwxyz
Good guess: _la__e
-------------
You have 8 guesses left.
Available letters: bcdfghijkmnopqrstuvwxyz
Oops! That letter is not in my word: _la__e
-------------
You have 7 guesses left.
Available letters: bdfghijkmnopqrstuvwxyz
Oops! That letter is not in my word: _la__e
-------------
You have 6 guesses left.
Available letters: bdfghijkmnoqrstuvwxyz
Oops! That letter is not in my word: _la__e
-------------
You have 5 guesses left.
Available letters: bdfghijkmnoqrstvwxyz
Oops! That letter is not in my word: _la__e
-------------
You have 4 guesses left.
Available letters

Oops! That letter is not in my word: ________
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 6 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Oops! That letter is not in my word: ________
-------------
You have 5 guesses left.
Available letters: abcdfghjklmnopqrtuvwxyz
Good guess: ____a_a_
-------------
You have 5 guesses left.
Available letters: bcdfghjklmnopqrtuvwxyz
Good guess: ____ana_
-------------
You have 5 guesses left.
Available letters: bcdfghjklmopqrtuvwxyz
Oops! That letter is not in my word: ____ana_
-------------
You have 4 guesses left.
Available letters: bcdfgjklmopqrtuvwxyz
Good guess: ___lana_
-------------
You have 4 guesses left.
Available letters: bcdfgjkmopqrtuvwxyz
Good guess: c__lana_
-------------
You have 4 guesses left.
Available letters: bdfgjkmopqrtuvwxyz
Good guess: co_lana_
-------------
You have 4 guesses left.
Available letters: bdfgjkmpq

Good guess: fair
-------------
Congratulations, you won!
Welcome to the game Hangman!
I am thinking of a word that is 9 letters long.
-------------
You have 8 guesses left.
Available letters: abcdefghijklmnopqrstuvwxyz
Good guess: ____e__e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ____e__e_
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqrtuvwxyz
Good guess: ____er_er
-------------
You have 7 guesses left.
Available letters: abcdfghijklmnopqtuvwxyz
Oops! That letter is not in my word: ____er_er
-------------
You have 6 guesses left.
Available letters: acdfghijklmnopqtuvwxyz
Good guess: ____erier
-------------
You have 6 guesses left.
Available letters: acdfghjklmnopqtuvwxyz
Good guess: f___erier
-------------
You have 6 guesses left.
Available letters: acdghjklmnopqtuvwxyz
Good guess: fl__erier
-------------
You have 6 guesses left.
Available letters: acdghjkmnopqtuvwxyz
Good guess

Good guess: ___e_e_
-------------
You have 8 guesses left.
Available letters: abcdfghijklmnopqrstuvwxyz
Good guess: ___e_ed
-------------
You have 8 guesses left.
Available letters: abcfghijklmnopqrstuvwxyz
Oops! That letter is not in my word: ___e_ed
-------------
You have 7 guesses left.
Available letters: abcfghijklmnopqstuvwxyz
Oops! That letter is not in my word: ___e_ed
-------------
You have 6 guesses left.
Available letters: abcfghijkmnopqstuvwxyz
Oops! That letter is not in my word: ___e_ed
-------------
You have 5 guesses left.
Available letters: abcfghijkmnopqsuvwxyz
Good guess: ___ened
-------------
You have 5 guesses left.
Available letters: abcfghijkmopqsuvwxyz
Good guess: _i_ened
-------------
You have 5 guesses left.
Available letters: abcfghjkmopqsuvwxyz
Good guess: wi_ened
-------------
You have 5 guesses left.
Available letters: abcfghjkmopqsuvxyz
Good guess: wizened
-------------
Congratulations, you won!
Player function won 96 games.
Player function lost 4 games.
P